In [73]:
import tensorflow as tf
import os

imgDir = os.path.join('dataset', 'flowers')
imgSize = 120

In [74]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(imgDir, image_size=(imgSize,imgSize), batch_size=32)
testData = dataset.take(25)
trainData = dataset.skip(25)

Found 13838 files belonging to 14 classes.


In [75]:
# build verification model
inputs = tf.keras.layers.Input(shape=(imgSize, imgSize, 3))
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(units=32, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(loss='sparse_categorical_crossentropy', metrics='sparse_categorical_accuracy', optimizer='adam')
model.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 120, 120, 3)]     0         
                                                                 
 conv2d_24 (Conv2D)          (None, 118, 118, 32)      896       
                                                                 
 conv2d_25 (Conv2D)          (None, 116, 116, 64)      18496     
                                                                 
 flatten_20 (Flatten)        (None, 861184)            0         
                                                                 
 dropout_3 (Dropout)         (None, 861184)            0         
                                                                 
 dense_20 (Dense)            (None, 32)                27557920  
                                                                 
Total params: 27,577,312
Trainable params: 27,577,312
Non-

In [80]:
print(trainData, '\n', testData, '\n', dataset, dataset.cardinality)
model.fit(trainData, epochs=5)

<SkipDataset element_spec=(TensorSpec(shape=(None, 120, 120, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))> 
 <TakeDataset element_spec=(TensorSpec(shape=(None, 120, 120, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))> 
 <BatchDataset element_spec=(TensorSpec(shape=(None, 120, 120, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))> <bound method DatasetV2.cardinality of <BatchDataset element_spec=(TensorSpec(shape=(None, 120, 120, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>>
Epoch 1/5
408/408 [==============================] - 43s 105ms/step - loss: 0.1411 - sparse_categorical_accuracy: 0.9725
Epoch 2/5
408/408 [==============================] - 43s 105ms/step - loss: 0.2011 - sparse_categorical_accuracy: 0.9574
Epoch 3/5
408/408 [==============================] - 45s 108ms/step - loss: 0.1247 - sparse_categorical_accuracy

In [81]:
model.evaluate(testData)

25/25 [==============================] - 1s 37ms/step - loss: 3.7141 - sparse_categorical_accuracy: 0.7150


[3.7141237258911133, 0.7149999737739563]